In [1]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append(os.path.abspath(".."))
pd.set_option("display.precision", 2)
from ficaria.missing_imputation import FCMCentroidImputer, FCMParameterImputer, FCMRoughParameterImputer

In [2]:
# Sample data
data = pd.DataFrame({
    'age': [25, 26, 28, 51, 53, 72, 75, np.nan],
    'income': [50000, 55000, 58000, 85000, 90000, 120000, 125000, 100000]
})


imputer = FCMCentroidImputer(n_clusters=3)
imputer.fit(data)
filled_data = imputer.transform(data)

print(filled_data)

    age  income
0  25.0   50000
1  26.0   55000
2  28.0   58000
3  51.0   85000
4  53.0   90000
5  72.0  120000
6  75.0  125000
7  52.0  100000


In [3]:
data = pd.DataFrame({
    'systolic_bp': [120, 130, 125, np.nan, 140, 135, np.nan, 150],
    'diastolic_bp': [80, 85, np.nan, 88, 90, 82, 95, 100],
    'heart_rate': [70, 72, 75, np.nan, 78, 80, 76, np.nan]
})

imputer = FCMParameterImputer(n_clusters=3)
imputer.fit(data)
filled_data = imputer.transform(data)

print(filled_data)

   systolic_bp  diastolic_bp  heart_rate
0       120.00         80.00       70.00
1       130.00         85.00       72.00
2       125.00         82.15       75.00
3       138.15         88.00       77.45
4       140.00         90.00       78.00
5       135.00         82.00       80.00
6       136.95         95.00       76.00
7       150.00        100.00       76.95


In [4]:
data = pd.DataFrame({
    'temperature': [21.5, 22.0, np.nan, 23.1, 21.8, 22.3, np.nan, 22.7],
    'humidity': [40, 42, 38, np.nan, 41, 43, 39, np.nan],
    'pressure': [1012, 1013, 1011, 1010, np.nan, 1014, 1012, np.nan]
})

imputer = FCMRoughParameterImputer(n_clusters=3, max_iter=1000)
imputer.fit(data)
filled_data = imputer.transform(data)
print(filled_data)

   temperature  humidity  pressure
0         21.5      40.0    1012.0
1         22.0      42.0    1013.0
2         21.5      38.0    1011.0
3         23.1      40.0    1010.0
4         21.8      41.0    1013.0
5         22.3      43.0    1014.0
6         21.5      39.0    1012.0
7         22.7      43.0    1014.0
